In [ ]:
import pandas as pd

def k_shingles(text, k):
    words = text.split()
    shingles = set()
    for i in range(len(words) - k + 1):
        shingle = ' '.join(words[i:i + k])
        shingles.add(shingle)
    return shingles

# Define the hash functions
def hash_function_1(x):
    return (3 * x + 5) % 268

def hash_function_2(x):
    return (7 * x + 4) % 268

def hash_function_3(x):
    return (3 * x + 1) % 268

# File paths
file_paths = [
    "/content/firwst.txt",
    "/content/sec.txt",
    "/content/third.txt"
]

# Token size
k = 5

# Dictionary to hold shingles for each file
shingle_dict = {}

# Process each file
for file_path in file_paths:
    with open(file_path, 'r') as file:
        content = file.read()

    shingles = k_shingles(content, k)
    shingle_dict[file_path] = shingles

# Create a set of all unique shingles across all files
all_shingles = list(set.union(*[set(shingles) for shingles in shingle_dict.values()]))

# Initialize an incidence matrix
incidence_matrix = pd.DataFrame(0, index=all_shingles, columns=file_paths)

# Populate the incidence matrix
for file_path, shingles in shingle_dict.items():
    for shingle in shingles:
        incidence_matrix.at[shingle, file_path] = 1

# Function to compute signatures
def compute_signature(incidence_matrix):
    signatures = {file_path: [] for file_path in incidence_matrix.columns}

    for shingle in incidence_matrix.index:
        # Use the index of the shingle in the all_shingles list
        index = list(incidence_matrix.index).index(shingle)

        for file_path in incidence_matrix.columns:
            # Calculate hash values
            hash_value = [
                hash_function_1(index),
                hash_function_2(index),
                hash_function_3(index)
            ]
            # Append hash values if the shingle is present in the file
            if incidence_matrix.at[shingle, file_path] == 1:
                signatures[file_path].append(hash_value)

    # Find the minimum hash value for each hash function per file
    final_signatures = {}
    for file_path, values in signatures.items():
        if values:  # If there are any values
            final_signatures[file_path] = [
                min(value[0] for value in values),  # Min for Hash1
                min(value[1] for value in values),  # Min for Hash2
                min(value[2] for value in values)   # Min for Hash3
            ]
        else:
            final_signatures[file_path] = [None, None, None]  # Handle case with no shingles

    return pd.DataFrame(final_signatures, index=['Hash1', 'Hash2', 'Hash3']).T

# Compute signatures
signature_matrix = compute_signature(incidence_matrix)

# Display the incidence matrix and signature matrix
print("Incidence Matrix:")
print(incidence_matrix)

print("\nSignature Matrix:")
print(signature_matrix)


Incidence Matrix:
                                 /content/firwst.txt  /content/sec.txt  \
that trees are our best                            1                 1   
our friends need saving. They                      0                 1   
changes that are destroying the                    0                 0   
benefit every life form in                         1                 1   
us and protect us in                               0                 1   
...                                              ...               ...   
From childhood, we have heard                      1                 1   
that we breathe and absorb                         0                 0   
from heat and sunlight, etc.                       0                 1   
the climatic changes that are                      0                 0   
protect us in many ways.                           0                 1   

                                 /content/third.txt  
that trees are our best                

Bloom Filter

In [ ]:
class BloomFilter:
    def __init__(self, size):
        self.size = size
        self.bit_array = [0] * size

    def hash1(self, x):
        return (x + 1) % self.size

    def hash2(self, x):
        return (2 * x + 5) % self.size

    def add(self, x):
        index1 = self.hash1(x)
        index2 = self.hash2(x)
        self.bit_array[index1] = 1
        self.bit_array[index2] = 1

    def check(self, x):
        index1 = self.hash1(x)
        index2 = self.hash2(x)
        return self.bit_array[index1] == 1 and self.bit_array[index2] == 1

# Initialize Bloom filter
bloom_filter = BloomFilter(13)

# Add elements 8, 17, 25, 14, 20 to the Bloom filter
elements_to_add = [8, 17, 25, 14, 20]
for elem in elements_to_add:
    bloom_filter.add(elem)

# Check for integers 7 and 5
check_elements = [7, 5]
for elem in check_elements:
    if bloom_filter.check(elem):
        print(f"Element {elem} may be in the set.")
    else:
        print(f"Element {elem} is definitely not in the set.")

Element 7 may be in the set.
Element 5 may be in the set.


ASM Algorithm

In [ ]:
#USING EXAMPLE
# AMS Algorithm Implementation
def ams_algorithm(stream, x_values):
    n = len(stream)

    # Initialize the sum of square estimates
    sum_squared_estimates = 0

    # Perform the AMS estimate for each x_value
    for x in x_values:
        # Choose a random element r from the stream
        r = stream[x - 1]  # x is 1-indexed, so we use x-1 for 0-indexed lists

        # Count the number of times r appears in the stream
        count_r = stream.count(r)

        # Calculate the square of count_r and update the sum of square estimates
        sum_squared_estimates += n * (2 * count_r - 1)

    # Return the average of the square estimates
    return sum_squared_estimates / len(x_values)

# Given stream and x values
stream = [2, 3, 7, 1, 5, 8, 5, 7, 9, 6, 4, 4, 5, 6, 5, 8, 8, 5, 2,2, 2, 1, 1, 6, 7]
x_values = [1, 3, 5, 10]  # Values of x1, x2, x3, and x4 as 1, 3, 5, and 10

# Calculate the surprise number using AMS algorithm
surprise_number = ams_algorithm(stream, x_values)

# Output the result
print("The surprise number (Second Frequency Moment Estimate) is:", surprise_number)

The surprise number (Second Frequency Moment Estimate) is: 162.5


In [ ]:
import random
# AMS Algorithm Implementation
def ams_algorithm(stream, x_values):
    n = len(stream)

    # Initialize the sum of square estimates
    sum_squared_estimates = 0

    # Perform the AMS estimate for each x_value
    for x in x_values:
        # Choose a random element r from the stream
        r = stream[x - 1]  # x is 1-indexed, so we use x-1 for 0-indexed lists

        # Count the number of times r appears in the stream
        count_r = stream.count(r)

        # Calculate the square of count_r and update the sum of square estimates
        sum_squared_estimates += n * (2 * count_r - 1)

    # Return the average of the square estimates
    return sum_squared_estimates / len(x_values)

# Generate a random stream of integers
random_stream = [random.randint(1, 10) for _ in range(25)]

# Given x values
x_values = [1, 3, 5, 10]  # Values of x1, x2, x3, and x4 as 1, 3, 5, and 10

# Calculate the surprise number using AMS algorithm on the random stream
surprise_number = ams_algorithm(random_stream, x_values)

# Output the result
print("The random stream is:", random_stream)
print("The surprise number (Second Frequency Moment Estimate) is:", surprise_number)


The random stream is: [10, 9, 5, 10, 8, 6, 5, 7, 9, 4, 5, 3, 6, 6, 5, 2, 3, 7, 3, 2, 8, 4, 1, 9, 9]
The surprise number (Second Frequency Moment Estimate) is: 100.0


Flajolet Martin

In [ ]:
#using example
def hash_value(x, a, b, m):
    # Hash function: (a*x + b) mod m
    return (a * x + b) % m

def tail_length(bin_str):
    # Find the length of trailing zeros in the binary representation of the hash value
    return len(bin_str) - len(bin_str.rstrip('0'))

def flajolet_martin(stream, a, b, m):
    max_tail_length = 0
    tail_lengths = []  # List to store tail lengths for each element

    for element in stream:
        # Hash the element and convert to binary
        hash_val = hash_value(element, a, b, m)
        bin_hash = format(hash_val, 'b')  # Convert to binary

        # Calculate the tail length of the binary hash
        t_len = tail_length(bin_hash)
        tail_lengths.append((element, hash_val, bin_hash, t_len))  # Store element, hash, and tail length

        # Keep track of the maximum tail length
        max_tail_length = max(max_tail_length, t_len)

    # Print the tail lengths
    for elem, h_val, bin_h, t_len in tail_lengths:
        print(f"Element: {elem}, Hash: {h_val}, Binary Hash: {bin_h}, Tail Length: {t_len}")

    # Estimate the number of distinct elements
    return 2 ** max_tail_length

# Example usage
stream = [3, 1, 4, 1, 5, 9, 2, 6, 5]

# Use hash function 3x + 7 mod 32
a = 3
b = 7
m = 32

estimate = flajolet_martin(stream, a, b, m)
print(f"\nEstimated number of distinct elements: {estimate}")

Element: 3, Hash: 16, Binary Hash: 10000, Tail Length: 4
Element: 1, Hash: 10, Binary Hash: 1010, Tail Length: 1
Element: 4, Hash: 19, Binary Hash: 10011, Tail Length: 0
Element: 1, Hash: 10, Binary Hash: 1010, Tail Length: 1
Element: 5, Hash: 22, Binary Hash: 10110, Tail Length: 1
Element: 9, Hash: 2, Binary Hash: 10, Tail Length: 1
Element: 2, Hash: 13, Binary Hash: 1101, Tail Length: 0
Element: 6, Hash: 25, Binary Hash: 11001, Tail Length: 0
Element: 5, Hash: 22, Binary Hash: 10110, Tail Length: 1

Estimated number of distinct elements: 16


In [ ]:
def hash_value(x, a, b, m):
    # Hash function: (a*x + b) mod m
    return (a * x + b) % m

def tail_length(bin_str):
    # Find the length of trailing zeros in the binary representation of the hash value
    return len(bin_str) - len(bin_str.rstrip('0'))

def flajolet_martin(stream, a, b, m):
    max_tail_length = 0
    tail_lengths = []  # List to store tail lengths for each element

    for element in stream:
        # Hash the element and convert to binary
        hash_val = hash_value(element, a, b, m)
        bin_hash = format(hash_val, 'b')  # Convert to binary

        # Calculate the tail length of the binary hash
        t_len = tail_length(bin_hash)
        tail_lengths.append((element, hash_val, bin_hash, t_len))  # Store element, hash, and tail length

        # Keep track of the maximum tail length
        max_tail_length = max(max_tail_length, t_len)

    # Print the tail lengths
    for elem, h_val, bin_h, t_len in tail_lengths:
        print(f"Element: {elem}, Hash: {h_val}, Binary Hash: {bin_h}, Tail Length: {t_len}")

    # Estimate the number of distinct elements
    return 2 ** max_tail_length

# Example usage
stream = [3, 1, 4, 1, 5, 9, 2, 6, 5,3,5]

# Use hash function 3x + 7 mod 32
a = 2
b = 1
m = 32

estimate = flajolet_martin(stream, a, b, m)
print(f"\nEstimated number of distinct elements: {estimate}")

Element: 3, Hash: 7, Binary Hash: 111, Tail Length: 0
Element: 1, Hash: 3, Binary Hash: 11, Tail Length: 0
Element: 4, Hash: 9, Binary Hash: 1001, Tail Length: 0
Element: 1, Hash: 3, Binary Hash: 11, Tail Length: 0
Element: 5, Hash: 11, Binary Hash: 1011, Tail Length: 0
Element: 9, Hash: 19, Binary Hash: 10011, Tail Length: 0
Element: 2, Hash: 5, Binary Hash: 101, Tail Length: 0
Element: 6, Hash: 13, Binary Hash: 1101, Tail Length: 0
Element: 5, Hash: 11, Binary Hash: 1011, Tail Length: 0
Element: 3, Hash: 7, Binary Hash: 111, Tail Length: 0
Element: 5, Hash: 11, Binary Hash: 1011, Tail Length: 0

Estimated number of distinct elements: 1
